# Iterative Random Shaking Volume Denoising (IRSVD)

In [ ]:
local_debug = True

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Colab")
    !pip install cupy-cuda12x
    !pip install opticalflow3D
    !apt install libcudart11.0
    !apt install libcublas11
    !apt install libcufft10
    !apt install libcusparse11
    !apt install libnvrtc11.2
    from google.colab import drive
    drive.mount('/content/drive')
    !cp drive/Shareddrives/TomogramDenoising/tomograms/{vol_name}.tif .
else:
    print("Running in locahost")
    !cp ~/Downloads/{vol_name}.tif .

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
import opticalflow3D
import warnings
from numba.core.errors import NumbaPerformanceWarning
import numpy as np

warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

In [ ]:
if local_debug:
    !ln -sf ../../information_theory/src/information_theory/ .
else:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
import information_theory  # pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
import skimage.io

In [ ]:
import RSIVD

In [ ]:
from collections import namedtuple
Args = namedtuple("args", ["input", "output"])
args = Args("/home/vruiz/cryoCARE/tomo2_L1G1_bin4.rec", "denoised")

In [ ]:
import mrcfile
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data#[:, ::-1, :]

%%bash -s "$args.input"
set -x
OUTPUT_FILENAME=$1
#rm -f $OUTPUT_FILENAME
if test ! -f $OUTPUT_FILENAME ; then
    FILEID="1I2uIfM00ZNeMjYy4OeZ4hSO-bxpE3oZb"
    gdown https://drive.google.com/uc?id=$FILEID
fi
set +x
# https://drive.google.com/file/d/1I2uIfM00ZNeMjYy4OeZ4hSO-bxpE3oZb/view?usp=sharing

noisy = opticalflow3D.helpers.load_image(args.input)

In [ ]:
np.min(noisy)

In [ ]:
np.max(noisy)

In [ ]:
noisy = (255*(noisy.astype(np.float32) - np.min(noisy))/(np.max(noisy) - np.min(noisy))).astype(np.uint8)

In [ ]:
np.min(noisy)

In [ ]:
np.max(noisy)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(16, 16))
axs.imshow(noisy[91], cmap="gray")
axs.set_title(f"Noisy")
fig.tight_layout()
plt.show()

In [ ]:
noisy.shape

In [ ]:
#noisy = noisy[100:300, 300:600, 300:600]

In [ ]:
#block_size = (noisy.shape[0], noisy.shape[1], noisy.shape[2])
block_size = (noisy.shape[0]//2, noisy.shape[1]//2, noisy.shape[2]//4)
print(block_size)

In [ ]:
#farneback = opticalflow3D.Farneback3D(iters=5, num_levels=3, scale=0.5, spatial_size=7, presmoothing=3, filter_type="gaussian", filter_size=7,); RS_sigma = 1.0
farneback = opticalflow3D.Farneback3D(iters=1, num_levels=1, scale=0.5, spatial_size=5, sigma_k=1.0, filter_type="gaussian", filter_size=11, presmoothing=None, device_id=0); RS_sigma = 0.50; N_iters=25
farneback = opticalflow3D.Farneback3D(iters=1, num_levels=1, scale=0.5, spatial_size=7, sigma_k=1.0, filter_type="gaussian", filter_size=7, presmoothing=None, device_id=0); RS_sigma = 0.75; N_iters=50
farneback = opticalflow3D.Farneback3D(iters=2, num_levels=1, scale=0.5, spatial_size=7, sigma_k=1.0, filter_type="gaussian", filter_size=7, presmoothing=None, device_id=0); RS_sigma = 0.75; N_iters=50
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=6, sigma_k=1.0, filter_type="gaussian", filter_size=6, presmoothing=None, device_id=0); RS_sigma = 1.25; N_iters=50
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=6, sigma_k=1.0, filter_type="gaussian", filter_size=6, presmoothing=None, device_id=0); RS_sigma = 1.25; N_iters=25
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=6, sigma_k=1.0, filter_type="gaussian", filter_size=5, presmoothing=None, device_id=0); RS_sigma = 1.25; N_iters=25
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=6, sigma_k=1.0, filter_type="gaussian", filter_size=7, presmoothing=None, device_id=0); RS_sigma = 1.5; N_iters=25
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=6, sigma_k=1.0, filter_type="gaussian", filter_size=6, presmoothing=3, device_id=0); RS_sigma = 1.25; N_iters=25
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=9, sigma_k=1.0, filter_type="gaussian", filter_size=7, presmoothing=None, device_id=0); RS_sigma = 1.75; N_iters=25
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=2, scale=0.5, spatial_size=9, sigma_k=1.0, filter_type="gaussian", filter_size=7, presmoothing=None, device_id=0); RS_sigma = 2.0; N_iters=10
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=9, sigma_k=1.0, filter_type="gaussian", filter_size=7, presmoothing=None, device_id=0); RS_sigma = 1.5; N_iters=25
farneback = opticalflow3D.Farneback3D(iters=5, num_levels=1, scale=0.5, spatial_size=9, sigma_k=1.0, filter_type="gaussian", filter_size=9, presmoothing=None, device_id=0); RS_sigma = 1.5; N_iters=25


In [ ]:
def randomize(vol, mean=0.0, std_dev=1.0):
  depth, height, width = vol.shape[:3]
  x_coords, y_coords, z_coords = np.meshgrid(range(width), range(height), range(depth))
  flattened_x_coords = x_coords.flatten()
  flattened_y_coords = y_coords.flatten()
  flattened_z_coords = z_coords.flatten()
  #print(np.max(flattened_z_coords), np.max(flattened_y_coords), np.max(flattened_x_coords))
  #print(flattened_x_coords.dtype)
  displacements_x = np.random.normal(mean, std_dev, flattened_x_coords.shape).astype(np.int32)
  displacements_y = np.random.normal(mean, std_dev, flattened_y_coords.shape).astype(np.int32)
  displacements_z = np.random.normal(mean, std_dev, flattened_z_coords.shape).astype(np.int32)
  #_d = 5
  #displacements_x = np.random.uniform(low=-_d, high=_d, size=flattened_x_coords.shape).astype(np.int32)
  #displacements_y = np.random.uniform(low=-_d, high=_d, size=flattened_y_coords.shape).astype(np.int32)
  #displacements_z = np.random.uniform(low=-_d, high=_d, size=flattened_z_coords.shape).astype(np.int32)
  print("min displacements", np.min(displacements_z), np.min(displacements_y), np.min(displacements_x))
  print("average abs(displacements)", np.average(np.abs(displacements_z)), np.average(np.abs(displacements_y)), np.average(np.abs(displacements_x)))
  print("max displacements", np.max(displacements_z), np.max(displacements_y), np.max(displacements_x))
  randomized_x_coords = flattened_x_coords + displacements_x
  randomized_y_coords = flattened_y_coords + displacements_y
  randomized_z_coords = flattened_z_coords + displacements_z
  #print("max displacements", np.max(randomized_z_coords), np.max(randomized_y_coords), np.max(randomized_x_coords))
  #randomized_x_coords = np.mod(randomized_x_coords, width)
  #randomized_y_coords = np.mod(randomized_y_coords, height)
  #randomized_z_coords = np.mod(randomized_z_coords, depth)
  randomized_x_coords = np.clip(randomized_x_coords, 0, width - 1) # Clip the randomized coordinates to stay within image bounds
  randomized_y_coords = np.clip(randomized_y_coords, 0, height - 1)
  randomized_z_coords = np.clip(randomized_z_coords, 0, depth - 1)
  #print(np.max(randomized_z_coords), np.max(randomized_y_coords), np.max(randomized_x_coords))
  #randomized_vol = np.ones_like(vol)*np.average(vol) #np.zeros_like(vol)
  randomized_vol = np.zeros_like(vol)
  #randomized_vol[...] = vol
  #randomized_vol[...] = 128
  randomized_vol[randomized_z_coords, randomized_y_coords, randomized_x_coords] = vol[flattened_z_coords, flattened_y_coords, flattened_x_coords]
  return randomized_vol

def _randomize(vol, max_distance=10):
    depth, height, width = image.shape[:3]
    #flow_x = np.random.normal(size=(height, width)) * max_distance
    #flow_y = np.random.normal(size=(height, width)) * max_distance
    flow_x = np.random.uniform(low=-1, high=1, size=(depth, height, width)) * max_distance
    flow_y = np.random.uniform(low=-1, high=1, size=(depth, height, width)) * max_distance
    flow_z = np.random.uniform(low=-1, high=1, size=(depth, height, width)) * max_distance
    #flow_x[...] = 0
    #flow_y[...] = 0
    #print(np.max(flow_x), np.min(flow_x), max_distance)
    flow = np.empty([height, width, 2], dtype=np.float32)
    flow[..., 0] = flow_y
    flow[..., 1] = flow_x
    print(np.max(flow), np.min(flow))
    randomized_image = motion_estimation.project(image, flow)
    return randomized_image.astype(np.uint8)

def shake(x, y, std_dev=1.0):
  displacements = np.random.normal(0, std_dev, len(x))
  #print(f"{np.min(displacements):.2f} {np.average(np.abs(displacements)):.2f} {np.max(displacements):.2f}", end=' ')
  return np.stack((y + displacements, x), axis=1)

def randomize(vol, mean=0.0, std_dev=1.0):
  print(vol.shape)
  print(std_dev)
  randomized_vol = np.empty_like(vol)
  
  # Randomization in X
  #values = np.arange(1, vol.shape[2]+1).astype(np.int32)
  values = np.arange(vol.shape[2]).astype(np.int32)
  for z in range(vol.shape[0]):
    print(z, end=' ', flush=True)
    for y in range(vol.shape[1]):
      #pairs = np.array(list(map(tuplify, values, range(len(values)))), dtype=np.int32)
      pairs = shake(values, np.arange(len(values)), std_dev).astype(np.int32)
      pairs = pairs[pairs[:, 0].argsort()]
      randomized_vol[z, y, values] = vol[z, y, pairs[:, 1]]
  vol = np.copy(randomized_vol)

  # Randomization in Y
  values = np.arange(vol.shape[1]).astype(np.int32)
  for z in range(vol.shape[0]):
    print(z, end=' ', flush=True)
    for x in range(vol.shape[2]):
      #pairs = np.array(list(map(tuplify, values, range(len(values)))), dtype=np.int32)
      pairs = shake(values, np.arange(len(values)), std_dev).astype(np.int32)
      pairs = pairs[pairs[:, 0].argsort()]
      randomized_vol[z, values, x] = vol[z, pairs[:, 1], x]
  vol = np.copy(randomized_vol)

  # Randomization in Z
  values = np.arange(vol.shape[0]).astype(np.int32)
  for y in range(vol.shape[1]):
    print(y, end=' ', flush=True)
    for x in range(vol.shape[2]):
      #pairs = np.array(list(map(tuplify, values, range(len(values)))), dtype=np.int32)
      pairs = shake(values, np.arange(len(values)), std_dev).astype(np.int32)
      pairs = pairs[pairs[:, 0].argsort()]
      randomized_vol[values, y, x] = vol[pairs[:, 1], y , x]

  return randomized_vol

def project_A_to_B(farneback, block_size, A, B):
  output_vz, output_vy, output_vx, output_confidence = farneback.calculate_flow(A, B,
                                                                              start_point=(0, 0, 0),
                                                                              total_vol=(A.shape[0], A.shape[1], A.shape[2]),
                                                                              sub_volume=block_size,
                                                                              overlap=(8, 8, 8),
                                                                              threadsperblock=(8, 8, 8)
                                                                             )
  print("min flow", np.min(output_vx), np.min(output_vy), np.min(output_vz))
  print("average abs(flow)", np.average(np.abs(output_vx)), np.average(np.abs(output_vy)), np.average(np.abs(output_vz)))
  print("max flow", np.max(output_vx), np.max(output_vy), np.max(output_vz))
  #output_vx[...] = 0
  #output_vy[...] = 0
  #output_vz[...] = 0
  projection = opticalflow3D.helpers.generate_inverse_image(A, output_vx, output_vy, output_vz, use_gpu=False)
  return projection

def filter(farneback, block_size, noisy_vol, N_iters=50, RS_sigma=2.0, RS_mean=0.0):
  acc_vol = np.zeros_like(noisy_vol, dtype=np.float32)
  acc_vol[...] = noisy_vol
  for i in range(N_iters):
    print(f"iter={i}")
    denoised_vol = acc_vol/(i+1)
    #randomized_noisy_vol = randomize(noisy_vol, max_distance=5)
    randomized_noisy_vol = randomize(noisy_vol, mean=0, std_dev=RS_sigma)
    #print("sum(randomized_noisy-noisy)", np.sum((randomized_noisy_vol-noisy_vol)*(randomized_noisy_vol-noisy_vol)))
    #print("sum(denoised-randomized_noisy)", np.sum((denoised_vol-randomized_noisy_vol)*(denoised_vol-randomized_noisy_vol)))
    randomized_and_compensated_noisy_vol = project_A_to_B(farneback, block_size, A=denoised_vol, B=randomized_noisy_vol)
    if i == 0:
        with mrcfile.new("shaked.mrc", overwrite=True) as mrc:
            mrc.set_data(randomized_noisy_vol.astype(np.float32))
            mrc.data
        with mrcfile.new("compensated.mrc", overwrite=True) as mrc:
            mrc.set_data(randomized_and_compensated_noisy_vol.astype(np.float32))
            mrc.data
    #plt.imshow(randomized_and_compensated_noisy_vol[15], cmap="gray")
    #plt.show()
    #randomized_and_compensated_noisy_vol = np.zeros_like(randomized_noisy_vol)
    #randomized_and_compensated_noisy_vol[...] = randomized_noisy_vol
    #print("sum(noisy)", np.sum(noisy_vol))
    #print("sum(denoised)", np.sum(denoised_vol))
    #print("sum(randomized_and_compensated_noisy)", np.sum(randomized_and_compensated_noisy_vol))
    #print("sum(randomized_noisy)", np.sum(randomized_noisy_vol))
    #print("sum(acc)", np.sum(acc_vol))
    #print("sum(randomized_and_compensated_noisy-randomized_noisy)", np.sum((randomized_and_compensated_noisy_vol-randomized_noisy_vol)*(randomized_and_compensated_noisy_vol-randomized_noisy_vol)))
    #print("sum(randomized_and_compensated_noisy-noisy)", np.sum((randomized_and_compensated_noisy_vol-noisy_vol)*(randomized_and_compensated_noisy_vol-noisy_vol)))
    #print(np.sum((randomized_and_compensated_noisy_vol-randomized_noisy_vol)*(randomized_and_compensated_noisy_vol-randomized_noisy_vol)))
    acc_vol += randomized_and_compensated_noisy_vol
  denoised_vol = acc_vol/(N_iters + 1)
  return denoised_vol

In [ ]:
#denoised = RSIVD.filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=N_iters)
denoised = filter(farneback, block_size, noisy, RS_sigma=RS_sigma, N_iters=N_iters)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[16], denoised[16])})")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[10], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(cryo[80+10, 150:300, 150:300], cmap="gray")
axs[1].set_title(f"cryoCARE (DQI={information_theory.information.compute_quality_index(noisy[10], cryo[80+10, 150:300, 150:300])})")
fig.tight_layout()
plt.show()

fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[131][300:,300:], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[131][300:,300:], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[131][300:,300:], denoised[131][300:,300:])}")
fig.tight_layout()
plt.show()

fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[:, 100], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, 100], cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[:, 100], denoised[:, 100])})")
fig.tight_layout()
plt.show()

figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy[137][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised[137][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[137][::-1, :] - denoised[137][::-1, :], cmap="gray")

In [ ]:
with mrcfile.new(f"{args.output}_{RS_sigma}_{N_iters}.mrc", overwrite=True) as mrc:
            mrc.set_data(denoised.astype(np.float32))
            mrc.data
skimage.io.imsave(f"{args.output}_{RS_sigma}_{N_iters}.tif", denoised, imagej=True)

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
import denoising.image.OF_random as denoising

In [ ]:
denoiser = denoising.Monochrome_Denoiser(
    logger,
    pyramid_levels = 3,
    window_side = 15,
    N_poly = 5,
    num_iterations = 10
)

In [ ]:
denoised, _ = denoiser.filter(noisy[10], None, N_iters=300, RS_sigma=0.5)

In [ ]:
print(np.min(denoised), np.max(denoised))

In [ ]:
denoised = np.clip(denoised, a_min=0, a_max=255)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
axs[0].imshow(noisy[10], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised, cmap="gray")
axs[1].set_title(f"Denoised (DQI={information_theory.information.compute_quality_index(noisy[10], denoised)})")
fig.tight_layout()
plt.show()

In [ ]:
input()

In [ ]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=3,
                                      scale=0.5,
                                      spatial_size=5,
                                      presmoothing=4,
                                      filter_type="box",
                                      filter_size=5,
                                     )

In [ ]:
RS_sigma = 1.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")

In [ ]:
skimage.io.imsave(f"{vol_name}_denoised_{RS_sigma}.tif", denoised_vol, imagej=True)

In [ ]:
RS_sigma = 2.0
denoised_vol = RSIVD.filter(farneback, block_size, noisy_vol, RS_sigma=RS_sigma, N_iters=25)

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
imgplot = plt.imshow(noisy_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
plt.title("$\sigma_\mathrm{RS}=$"+f"{RS_sigma}")
plt.imshow(denoised_vol[75][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy_vol[75][::-1, :] - denoised_vol[75][::-1, :], cmap="gray")